# Lysozyme + Mutations tutorial using BIOBB
Based on the official Gromacs tutorial: http://www.mdtutorials.com/gmx/lysozyme/01_pdb2gmx.html

In [1]:
import nglview
import ipywidgets

## Dowload the initial Structure
Download the initial PDB file from a PDB code (1AKI) from the RCSB (https://files.rcsb.org/download/).
The PDB will be dowloaded without solvent or HETATM.

In [2]:
#Import module
from biobb_io.api.pdb import Pdb
# Create properties dict and inputs/outputs
prop = {'pdb_code':'1aki'}
downloaded_pdb_path = '1aki.pdb'
#Create and launch bb
Pdb(output_pdb_path=downloaded_pdb_path, properties=prop).launch()

2019-02-25 13:14:43,844 [MainThread  ] [INFO ]  Downloading: 1aki from: https://files.rcsb.org/download/1aki.pdb
2019-02-25 13:14:44,303 [MainThread  ] [INFO ]  Writting pdb to: /home/stain/src/biobb_workflows/biobb_workflows/lysozyme_tutorial/notebooks/1aki.pdb
2019-02-25 13:14:44,304 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'MODEL', 'ENDMDL']


In [3]:
#Show protein
view = nglview.show_file(downloaded_pdb_path)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

## Modeling the missing heavy atoms in the structure side chains
This BB will reconstruct missing atoms from residue side chains and detect all kinds of clashes

In [4]:
#Check & Fix PDB
#Import module
from biobb_model.model.fix_side_chain import FixSideChain
# Create prop dict and inputs/outputs
fixed_pdb_path = '1aki.pdb'
#Create and launch bb
FixSideChain(input_pdb_path=downloaded_pdb_path, output_pdb_path=fixed_pdb_path).launch()

=                     MDWeb structure checking utility                        =
=                      A. Hospital, J.L. Gelpi 2018                           =

Running fixside. Options: --fix All 
1aki.pdb
Structure 1aki.pdb loaded
Title: 
Experimental method: unknown
Resolution: 0.0 A

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  129
 Num. residues with ins. codes:  0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  1001

No residues with missing side chain atoms found
Structure not modified, saving due to --force_save option
Structure saved on 1aki_fixed.pdb
Final Num. models: 1
Final Num. chains: 1 (A: Protein)
Final Num. residues:  129
Final Num. residues with ins. codes:  0
Final Num. HETATM residues:  0
Final Num. ligands or modified residues:  0
Final Num. water mol.:  0
Final Num. atoms:  1001



In [5]:
#Show protein
view = nglview.show_file(fixed_pdb_path)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

## Modeling mutation list
This BB will mutate existing residues to the requested ones.

In [6]:
#Mutate protein
# Import module to test
from biobb_model.model.mutate import Mutate
# Create prop dict and inputs/outputs
prop = {'mutation_list': 'Val2Tyr'}
mutated_pdb_path = '1aki_mutated.pdb'
#Create and launch bb
Mutate(input_pdb_path=fixed_pdb_path,output_pdb_path=mutated_pdb_path, properties=prop).launch()

=                     MDWeb structure checking utility                        =
=                      A. Hospital, J.L. Gelpi 2018                           =

Running mutateside. Options: --mut Val2Tyr 
1aki_fixed.pdb
Structure 1aki_fixed.pdb loaded
Title: 
Experimental method: unknown
Resolution: 0.0 A

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  129
 Num. residues with ins. codes:  0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  1001

Mutations to perform
*:VAL2TYR
Replacing VAL A2 into TYR
  Renaming CG1 to CG
  Deleting CG2
  Adding new atom CD1
  Adding new atom CE1
  Adding new atom CZ
  Adding new atom OH
  Adding new atom CE2
  Adding new atom CD2

Checking new clashes
No severe clashes detected
1 Steric apolar clashes detected
 TYR A2.CE2   PHE A38.CB      2.767 A
No polar_acceptor clashes detected
No polar_donor clashes detected
No positive clashes detected
No negative clashes detected
Structure saved o

In [8]:
#Show log and protein
view1 = nglview.show_file(fixed_pdb_path)
view1.add_representation(repr_type='ball+stick', selection='2:A')
view1._remote_call('setSize', target='Widget', args=['500px','600px'])
view1.camera='orthographic'
view2 = nglview.show_file(mutated_pdb_path)
view2.add_representation(repr_type='ball+stick', selection='2:A')
view2._remote_call('setSize', target='Widget', args=['500px','600px'])
view2.camera='orthographic'
ipywidgets.HBox([view1, view2])

## Generate the topology
Generate the topology using [Pdb2gmx](http://manual.gromacs.org/programs/gmx-pdb2gmx.html) module.
The default forcefield is [amber99sb-ildn](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2970904/) and the default water_type is [spce](http://www.sklogwiki.org/SklogWiki/index.php/SPC/E_model_of_water).
This BB will generate 2 main files:
-  A compressed ZIP file containing:
> -  The [Gromacs topology file](http://manual.gromacs.org/online/top.html) (.top)
> -  The [Gromacs position restraint file](http://manual.gromacs.org/online/itp.html) (.itp)
-  A post-processed [Gromacs structure file](http://manual.gromacs.org/current/online/gro.html) (.gro)

In [9]:
#Create system topology
# Import module
from biobb_md.gromacs.pdb2gmx import Pdb2gmx
# Create inputs/outputs
output_pdb2gmx_gro_path = '1aki_pdb2gmx.gro'
output_pdb2gmx_top_zip_path = '1aki_pdb2gmx_top.zip'
#Create and launch bb
Pdb2gmx(input_pdb_path=mutated_pdb_path, output_gro_path=output_pdb2gmx_gro_path, output_top_zip_path=output_pdb2gmx_top_zip_path).launch()

2019-02-25 13:15:48,269 [MainThread  ] [INFO ]  GROMACS Pdb2gmx 20191 version detected
2019-02-25 13:15:48,634 [MainThread  ] [INFO ]  gmx pdb2gmx -f 1aki_mutated.pdb -o 1aki_pdb2gmx.gro -p p2g.top -water spce -ff amber99sb-ildn -i p2g.itp

2019-02-25 13:15:48,636 [MainThread  ] [INFO ]  Exit code 0

2019-02-25 13:15:48,637 [MainThread  ] [INFO ]  
Using the Amber99sb-ildn force field in directory amber99sb-ildn.ff

going to rename amber99sb-ildn.ff/aminoacids.r2b
going to rename amber99sb-ildn.ff/dna.r2b
going to rename amber99sb-ildn.ff/rna.r2b
Reading 1aki_mutated.pdb...
Read '', 1006 atoms
Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.
There are 1 chains and 0 blocks of water and 129 residues with 1006 atoms

  chain  #res #atoms
  1 'A'   129   1006  

Reading residue database... (Amber99sb-ildn)
Processing chain 1 'A' (1006 atoms, 129 residues)
Identified residue LYS1 as a starting terminus.
Identified residue LEU129 as a ending terminus.


0

In [10]:
#Show protein
view = nglview.show_file(output_pdb2gmx_gro_path)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

Note that hydrogen atoms had been added to the structure.
***

## Create the solvent box
Create the solvent box using the [Editconf](http://manual.gromacs.org/programs/gmx-editconf.html) module. The box will be centered, the distance between the solute and the box will be 1.0nm and the box shape will be cubic by default. The main output of this BB will be an updated post-procesed Gromacs structure file (.gro). 

In [11]:
# Editconf: Create solvent box
# Import module
from biobb_md.gromacs.editconf import Editconf
# Create prop dict and inputs/outputs
output_editconf_gro_path = '1aki_editconf.gro'
#Create and launch bb
Editconf(input_gro_path=output_pdb2gmx_gro_path, output_gro_path=output_editconf_gro_path).launch()

2019-02-25 13:16:28,573 [MainThread  ] [INFO ]  GROMACS Editconf 20191 version detected
2019-02-25 13:16:28,577 [MainThread  ] [INFO ]  Centering molecule in the box.
2019-02-25 13:16:28,578 [MainThread  ] [INFO ]  Distance of the box to molecule:   1.00
2019-02-25 13:16:28,579 [MainThread  ] [INFO ]  Box type: cubic
2019-02-25 13:16:28,609 [MainThread  ] [INFO ]  gmx editconf -f 1aki_pdb2gmx.gro -o 1aki_editconf.gro -d 1.0 -bt cubic -c

2019-02-25 13:16:28,611 [MainThread  ] [INFO ]  Exit code 0

2019-02-25 13:16:28,612 [MainThread  ] [INFO ]  Note that major changes are planned in future for editconf, to improve usability and utility.Read 1965 atoms
Volume: 55.8378 nm^3, corresponds to roughly 25100 electrons
No velocities found
    system size :  3.817  4.234  3.454 (nm)
    diameter    :  5.010               (nm)
    center      :  2.781  2.484  0.015 (nm)
    box vectors :  3.817  4.235  3.455 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  :  55.84          

0

## Fill the solvent box with water molecules
Fill the solvent box using the Gromacs Solvate module. The "spc216.gro" will be default solvent model. The main output of this BB will be an updated post-procesed Gromacs structure file (.gro) and a zip file containing the updated topology file (.top) and the restriction files (.itp).

In [12]:
# Solvate: Fill the box with water molecules
from biobb_md.gromacs.solvate import Solvate
# Create prop dict and inputs/outputs
output_solvate_gro_path = '1aki_solvate.gro'
output_solvate_top_zip_path = '1aki_solvate_top.zip'
#Create and launch bb
Solvate(input_solute_gro_path=output_editconf_gro_path, output_gro_path=output_solvate_gro_path, input_top_zip_path=output_pdb2gmx_top_zip_path, output_top_zip_path=output_solvate_top_zip_path).launch()

2019-02-25 13:17:11,321 [MainThread  ] [INFO ]  GROMACS Solvate 20191 version detected
2019-02-25 13:17:11,327 [MainThread  ] [INFO ]  Extracting: /home/stain/src/biobb_workflows/biobb_workflows/lysozyme_tutorial/notebooks/1aki_pdb2gmx_top.zip
2019-02-25 13:17:11,328 [MainThread  ] [INFO ]  to:
2019-02-25 13:17:11,329 [MainThread  ] [INFO ]  ['fb2abcee-c06c-458f-8373-4430ba42e7a8/p2g.itp', 'fb2abcee-c06c-458f-8373-4430ba42e7a8/p2g.top']
2019-02-25 13:17:11,331 [MainThread  ] [INFO ]  Unzipping: 
2019-02-25 13:17:11,332 [MainThread  ] [INFO ]  1aki_pdb2gmx_top.zip
2019-02-25 13:17:11,333 [MainThread  ] [INFO ]  To: 
2019-02-25 13:17:11,334 [MainThread  ] [INFO ]  fb2abcee-c06c-458f-8373-4430ba42e7a8/p2g.itp
2019-02-25 13:17:11,336 [MainThread  ] [INFO ]  fb2abcee-c06c-458f-8373-4430ba42e7a8/p2g.top
2019-02-25 13:17:11,793 [MainThread  ] [INFO ]  gmx solvate -cp 1aki_editconf.gro -cs spc216.gro -o 1aki_solvate.gro -p fb2abcee-c06c-458f-8373-4430ba42e7a8/p2g.top

2019-02-25 13:17:11,794 [

0

In [14]:
#Show protein
view = nglview.show_file(output_solvate_gro_path)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='green')
view.add_representation(repr_type='ball+stick', selection='SOL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

## Preprocess ion generation
Create the [portable binary run file (.tpr)](http://manual.gromacs.org/online/tpr.html) for ion generation, using the Grompp module. The main default parameters for this execution are:
-  integrator  = steep         ; Algorithm (steep = steepest descent minimization)
-  emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
-  emstep      = 0.01          ; Minimization step size
-  nsteps      = 50000         ; Maximum number of (minimization) steps to perform

The main output of this BB will be the portable binary run file (.tpr).

In [15]:
# Grompp: Creating portable binary run file for ion generation
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'minimization', 'nsteps':'50000'}}
output_gppion_tpr_path = '1aki_gppion.tpr'
#Create and launch bb
Grompp(input_gro_path=output_solvate_gro_path, input_top_zip_path=output_solvate_top_zip_path, output_tpr_path=output_gppion_tpr_path,  properties=prop).launch()

2019-02-25 13:17:40,310 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2019-02-25 13:17:40,311 [MainThread  ] [INFO ]  Will run a minimization md of 50000 steps
2019-02-25 13:17:40,316 [MainThread  ] [INFO ]  Extracting: /home/stain/src/biobb_workflows/biobb_workflows/lysozyme_tutorial/notebooks/1aki_solvate_top.zip
2019-02-25 13:17:40,317 [MainThread  ] [INFO ]  to:
2019-02-25 13:17:40,318 [MainThread  ] [INFO ]  ['b010203d-5b45-45eb-af9f-77fd2ad14b95/p2g.itp', 'b010203d-5b45-45eb-af9f-77fd2ad14b95/p2g.top']
2019-02-25 13:17:40,320 [MainThread  ] [INFO ]  Unzipping: 
2019-02-25 13:17:40,322 [MainThread  ] [INFO ]  1aki_solvate_top.zip
2019-02-25 13:17:40,322 [MainThread  ] [INFO ]  To: 
2019-02-25 13:17:40,323 [MainThread  ] [INFO ]  b010203d-5b45-45eb-af9f-77fd2ad14b95/p2g.itp
2019-02-25 13:17:40,324 [MainThread  ] [INFO ]  b010203d-5b45-45eb-af9f-77fd2ad14b95/p2g.top
2019-02-25 13:17:40,668 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1aki_solvate.gro -r 1aki_s

0

## Ion generation
Replace solvent molecules to neutralice the system and then reach a 0.05 mol/litre concentration by default. Using the Genion module.
The main output of this BB will be an updated post-procesed Gromacs structure file (.gro) and a zip file containing the updated topology file (.top) and the restriction files (.itp).

In [16]:
# Genion: Adding ions to reach a 0.05 nm concentration
from biobb_md.gromacs.genion import Genion
# Create prop dict and inputs/outputs
prop={'neutral':True, 'concentration':0.05}
output_genion_gro_path = '1aki_genion.gro'
output_genion_top_zip_path = '1aki_genion_top.zip'
#Create and launch bb
Genion(input_tpr_path=output_gppion_tpr_path, output_gro_path=output_genion_gro_path, input_top_zip_path=output_solvate_top_zip_path, output_top_zip_path=output_genion_top_zip_path, properties=prop).launch()

2019-02-25 13:17:45,321 [MainThread  ] [INFO ]  GROMACS Genion 20191 version detected
2019-02-25 13:17:45,322 [MainThread  ] [INFO ]  To reach up 0.05 mol/litre concentration
2019-02-25 13:17:45,327 [MainThread  ] [INFO ]  Extracting: /home/stain/src/biobb_workflows/biobb_workflows/lysozyme_tutorial/notebooks/1aki_solvate_top.zip
2019-02-25 13:17:45,328 [MainThread  ] [INFO ]  to:
2019-02-25 13:17:45,328 [MainThread  ] [INFO ]  ['ee394858-f21d-428e-8c6a-df01f983c4ce/p2g.itp', 'ee394858-f21d-428e-8c6a-df01f983c4ce/p2g.top']
2019-02-25 13:17:45,329 [MainThread  ] [INFO ]  Unzipping: 
2019-02-25 13:17:45,329 [MainThread  ] [INFO ]  1aki_solvate_top.zip
2019-02-25 13:17:45,330 [MainThread  ] [INFO ]  To: 
2019-02-25 13:17:45,330 [MainThread  ] [INFO ]  ee394858-f21d-428e-8c6a-df01f983c4ce/p2g.itp
2019-02-25 13:17:45,331 [MainThread  ] [INFO ]  ee394858-f21d-428e-8c6a-df01f983c4ce/p2g.top
2019-02-25 13:17:45,455 [MainThread  ] [INFO ]  echo "SOL" | gmx genion -s 1aki_gppion.tpr -o 1aki_geni

0

In [17]:
#Show protein
view = nglview.show_file(output_genion_gro_path)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='ball+stick', selection='NA')
view.add_representation(repr_type='ball+stick', selection='CL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

## Preprocess energy minimization
Create the [portable binary run file (.tpr)](http://manual.gromacs.org/online/tpr.html) for energy minimization, using the Grompp module. The main default parameters for this execution are:
-  integrator  = steep         ; Algorithm (steep = steepest descent minimization)
-  emtol       = 500.0         ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
-  emstep      = 0.01          ; Minimization step size
-  nsteps      = 50000         ; Maximum number of (minimization) steps to perform

The main output of this BB will be the portable binary run file (.tpr).

In [18]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'minimization', 'nsteps':'50000', 'emtol':'500'}}
output_gppmin_tpr_path = '1aki_gppmin.tpr'
#Create and launch bb
Grompp(input_gro_path=output_genion_gro_path, input_top_zip_path=output_genion_top_zip_path, output_tpr_path=output_gppmin_tpr_path,  properties=prop).launch()

2019-02-25 13:17:54,238 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2019-02-25 13:17:54,240 [MainThread  ] [INFO ]  Will run a minimization md of 50000 steps
2019-02-25 13:17:54,245 [MainThread  ] [INFO ]  Extracting: /home/stain/src/biobb_workflows/biobb_workflows/lysozyme_tutorial/notebooks/1aki_genion_top.zip
2019-02-25 13:17:54,246 [MainThread  ] [INFO ]  to:
2019-02-25 13:17:54,248 [MainThread  ] [INFO ]  ['524d61e0-40ff-450a-8d34-3cd0eb6980ee/p2g.itp', '524d61e0-40ff-450a-8d34-3cd0eb6980ee/p2g.top']
2019-02-25 13:17:54,249 [MainThread  ] [INFO ]  Unzipping: 
2019-02-25 13:17:54,251 [MainThread  ] [INFO ]  1aki_genion_top.zip
2019-02-25 13:17:54,252 [MainThread  ] [INFO ]  To: 
2019-02-25 13:17:54,254 [MainThread  ] [INFO ]  524d61e0-40ff-450a-8d34-3cd0eb6980ee/p2g.itp
2019-02-25 13:17:54,254 [MainThread  ] [INFO ]  524d61e0-40ff-450a-8d34-3cd0eb6980ee/p2g.top
2019-02-25 13:17:54,629 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1aki_genion.gro -r 1aki_geni

0

## Execute system equilibration
Execute the energy minimization using the MDrun module and the input the portable binary run file (.tpr) as the main input.
The main output of this BB will be updated structure file (.gro) and the [Gromacs trajectory file](http://manual.gromacs.org/online/trr.html) (.trr).

In [19]:
# Mdrun: Running minimization
from biobb_md.gromacs.mdrun import Mdrun
# Create prop dict and inputs/outputs
output_min_trr_path = '1aki_min.trr'
output_min_gro_path = '1aki_min.gro'
output_min_edr_path = '1aki_min.edr'
output_min_log_path = '1aki_min.log'
#Create and launch bb
Mdrun(input_tpr_path=output_gppmin_tpr_path, output_trr_path=output_min_trr_path, output_gro_path=output_min_gro_path, output_edr_path=output_min_edr_path, output_log_path=output_min_log_path).launch()

2019-02-25 13:18:04,367 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2019-02-25 13:20:21,707 [MainThread  ] [INFO ]  gmx mdrun -s 1aki_gppmin.tpr -o 1aki_min.trr -c 1aki_min.gro -e 1aki_min.edr -g 1aki_min.log -nt 0

2019-02-25 13:20:21,708 [MainThread  ] [INFO ]  Exit code 0

2019-02-25 13:20:21,709 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    P

0

## Preprocess system temperature equilibration
Equilibrate the solvent and ions restraining the protein heavy atoms
Create the [portable binary run file (.tpr)](http://manual.gromacs.org/online/tpr.html) for system equilibration, using the Grompp module. The main default parameters for this execution are:
-  Define                   = -DPOSRES
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 5000
-  pcoupl                   = no
-  gen_vel                  = yes
-  gen_temp                 = 300
-  gen_seed                 = -1

The main output of this BB will be the portable binary run file (.tpr).

In [20]:
# Grompp: Creating portable binary run file for Equilibration
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'nvt', 'nsteps':'5000'}}
output_gppnvt_tpr_path = '1aki_gppnvt.tpr'
#Create and launch bb
Grompp(input_gro_path=output_min_gro_path, input_top_zip_path=output_genion_top_zip_path, output_tpr_path=output_gppnvt_tpr_path,  properties=prop).launch()

2019-02-25 13:22:55,578 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2019-02-25 13:22:55,580 [MainThread  ] [INFO ]  Will run a nvt md of 5000 steps
2019-02-25 13:22:55,585 [MainThread  ] [INFO ]  Extracting: /home/stain/src/biobb_workflows/biobb_workflows/lysozyme_tutorial/notebooks/1aki_genion_top.zip
2019-02-25 13:22:55,585 [MainThread  ] [INFO ]  to:
2019-02-25 13:22:55,586 [MainThread  ] [INFO ]  ['746dae26-1ae5-48a7-bfad-0219e3b3708c/p2g.itp', '746dae26-1ae5-48a7-bfad-0219e3b3708c/p2g.top']
2019-02-25 13:22:55,588 [MainThread  ] [INFO ]  Unzipping: 
2019-02-25 13:22:55,594 [MainThread  ] [INFO ]  1aki_genion_top.zip
2019-02-25 13:22:55,595 [MainThread  ] [INFO ]  To: 
2019-02-25 13:22:55,596 [MainThread  ] [INFO ]  746dae26-1ae5-48a7-bfad-0219e3b3708c/p2g.itp
2019-02-25 13:22:55,597 [MainThread  ] [INFO ]  746dae26-1ae5-48a7-bfad-0219e3b3708c/p2g.top
2019-02-25 13:22:56,168 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1aki_min.gro -r 1aki_min.gro -p 746dae

0

## Execute system temperature equilibration
Execute the system equilibration using the MDrun module and the input the portable binary run file (.tpr) as the main input.
The main output of this BB will be updated structure file (.gro) and the [Gromacs trajectory file](http://manual.gromacs.org/online/trr.html) (.trr).

In [21]:
# Mdrun: Running Equilibration NVT
from biobb_md.gromacs.mdrun import Mdrun
# Create prop dict and inputs/outputs
output_nvt_trr_path = '1aki_nvt.trr'
output_nvt_gro_path = '1aki_nvt.gro'
output_nvt_edr_path = '1aki_nvt.edr'
output_nvt_log_path = '1aki_nvt.log'
output_nvt_cpt_path = '1aki_nvt.cpt'
#Create and launch bb
Mdrun(input_tpr_path=output_gppnvt_tpr_path, output_trr_path=output_nvt_trr_path, output_gro_path=output_nvt_gro_path, output_edr_path=output_nvt_edr_path, output_log_path=output_nvt_log_path, output_cpt_path=output_nvt_cpt_path).launch()

2019-02-25 13:23:10,643 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2019-02-25 13:25:33,645 [MainThread  ] [INFO ]  gmx mdrun -s 1aki_gppnvt.tpr -o 1aki_nvt.trr -c 1aki_nvt.gro -e 1aki_nvt.edr -g 1aki_nvt.log -nt 0 -cpo 1aki_nvt.cpt

2019-02-25 13:25:33,647 [MainThread  ] [INFO ]  Exit code 0

2019-02-25 13:25:33,647 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik

0

## Preprocess system pressure equilibration
Equilibrate the solvent and ions restraining the protein heavy atoms.
Create the [portable binary run file (.tpr)](http://manual.gromacs.org/online/tpr.html) for system equilibration, using the Grompp module. The main default parameters for this execution are:
-  Define                   = -DPOSRES
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 5000
-  pcoupl = Parrinello-Rahman
-  pcoupltype = isotropic
-  tau_p = 1.0
-  ref_p = 1.0
-  compressibility = 4.5e-5
-  refcoord_scaling = com
-  gen_vel = no

The main output of this BB will be the portable binary run file (.tpr).

In [23]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'npt', 'nsteps':'5000'}}
output_gppnpt_tpr_path = '1aki_gppnpt.tpr'
#Create and launch bb
Grompp(input_gro_path=output_nvt_gro_path, input_top_zip_path=output_genion_top_zip_path, output_tpr_path=output_gppnpt_tpr_path, input_cpt_path=output_nvt_cpt_path,  properties=prop).launch()

2019-02-25 13:44:06,664 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2019-02-25 13:44:06,668 [MainThread  ] [INFO ]  Will run a npt md of 5000 steps
2019-02-25 13:44:06,674 [MainThread  ] [INFO ]  Extracting: /home/stain/src/biobb_workflows/biobb_workflows/lysozyme_tutorial/notebooks/1aki_genion_top.zip
2019-02-25 13:44:06,676 [MainThread  ] [INFO ]  to:
2019-02-25 13:44:06,678 [MainThread  ] [INFO ]  ['f13e9415-069e-4ae1-91ef-2472249f2b1c/p2g.itp', 'f13e9415-069e-4ae1-91ef-2472249f2b1c/p2g.top']
2019-02-25 13:44:06,680 [MainThread  ] [INFO ]  Unzipping: 
2019-02-25 13:44:06,680 [MainThread  ] [INFO ]  1aki_genion_top.zip
2019-02-25 13:44:06,681 [MainThread  ] [INFO ]  To: 
2019-02-25 13:44:06,681 [MainThread  ] [INFO ]  f13e9415-069e-4ae1-91ef-2472249f2b1c/p2g.itp
2019-02-25 13:44:06,682 [MainThread  ] [INFO ]  f13e9415-069e-4ae1-91ef-2472249f2b1c/p2g.top
2019-02-25 13:44:07,267 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1aki_nvt.gro -r 1aki_nvt.gro -p f13e94

0

## Execute system pressure equilibration
Execute the system equilibration using the MDrun module and the input the portable binary run file (.tpr) as the main input.
The main output of this BB will be updated structure file (.gro) and the [Gromacs trajectory file](http://manual.gromacs.org/online/trr.html) (.trr).

In [24]:
# Mdrun: Running minimization NPT
from biobb_md.gromacs.mdrun import Mdrun
# Create prop dict and inputs/outputs
output_npt_trr_path = '1aki_npt.trr'
output_npt_gro_path = '1aki_npt.gro'
output_npt_edr_path = '1aki_npt.edr'
output_npt_log_path = '1aki_npt.log'
output_npt_cpt_path = '1aki_npt.cpt'
#Create and launch bb
Mdrun(input_tpr_path=output_gppnpt_tpr_path, output_trr_path=output_npt_trr_path, output_gro_path=output_npt_gro_path, output_edr_path=output_npt_edr_path, output_log_path=output_npt_log_path, output_cpt_path=output_npt_cpt_path).launch()

2019-02-25 13:44:25,305 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2019-02-25 13:47:06,282 [MainThread  ] [INFO ]  gmx mdrun -s 1aki_gppnpt.tpr -o 1aki_npt.trr -c 1aki_npt.gro -e 1aki_npt.edr -g 1aki_npt.log -nt 0 -cpo 1aki_npt.cpt

2019-02-25 13:47:06,283 [MainThread  ] [INFO ]  Exit code 0

2019-02-25 13:47:06,283 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik

0

## Preprocess free dynamics
Free molecular dynamics simulation.
Create the [portable binary run file (.tpr)](http://manual.gromacs.org/online/tpr.html) for free molecular dynamics simulation, using the Grompp module. The main default parameters for this execution are:
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 5000

The main output of this BB will be the portable binary run file (.tpr).

In [25]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'free', 'nsteps':'15000'}}
output_gppmd_tpr_path = '1aki_gppmd.tpr'
#Create and launch bb
Grompp(input_gro_path=output_npt_gro_path, input_top_zip_path=output_genion_top_zip_path, output_tpr_path=output_gppmd_tpr_path, input_cpt_path=output_npt_cpt_path, properties=prop).launch()

2019-02-25 13:59:10,840 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2019-02-25 13:59:10,843 [MainThread  ] [INFO ]  Will run a free md of 30.0 pico seconds
2019-02-25 13:59:10,849 [MainThread  ] [INFO ]  Extracting: /home/stain/src/biobb_workflows/biobb_workflows/lysozyme_tutorial/notebooks/1aki_genion_top.zip
2019-02-25 13:59:10,850 [MainThread  ] [INFO ]  to:
2019-02-25 13:59:10,851 [MainThread  ] [INFO ]  ['e9a1f4d7-08a8-413b-979c-a12450006da0/p2g.itp', 'e9a1f4d7-08a8-413b-979c-a12450006da0/p2g.top']
2019-02-25 13:59:10,852 [MainThread  ] [INFO ]  Unzipping: 
2019-02-25 13:59:10,855 [MainThread  ] [INFO ]  1aki_genion_top.zip
2019-02-25 13:59:10,858 [MainThread  ] [INFO ]  To: 
2019-02-25 13:59:10,859 [MainThread  ] [INFO ]  e9a1f4d7-08a8-413b-979c-a12450006da0/p2g.itp
2019-02-25 13:59:10,860 [MainThread  ] [INFO ]  e9a1f4d7-08a8-413b-979c-a12450006da0/p2g.top
2019-02-25 13:59:11,272 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1aki_npt.gro -r 1aki_npt.gro -

0

## Execute free molecular dynamics simulation
Execute the free molecular dynamics simulation  using the MDrun module and the input the portable binary run file (.tpr) as the main input.
The main output of this BB will be updated structure file (.gro) and the [Gromacs trajectory file](http://manual.gromacs.org/online/trr.html) (.trr).

In [26]:
# Mdrun: Running free dynamics
from biobb_md.gromacs.mdrun import Mdrun
# Create prop dict and inputs/outputs
output_md_trr_path = '1aki_md.trr'
output_md_gro_path = '1aki_md.gro'
output_md_edr_path = '1aki_md.edr'
output_md_log_path = '1aki_md.log'
output_md_cpt_path = '1aki_md.cpt'
#Create and launch bb
Mdrun(input_tpr_path=output_gppmd_tpr_path, output_trr_path=output_md_trr_path, output_gro_path=output_md_gro_path, output_edr_path=output_md_edr_path, output_log_path=output_md_log_path, output_cpt_path=output_md_cpt_path).launch()

2019-02-25 13:59:18,388 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2019-02-25 14:08:21,993 [MainThread  ] [INFO ]  gmx mdrun -s 1aki_gppmd.tpr -o 1aki_md.trr -c 1aki_md.gro -e 1aki_md.edr -g 1aki_md.log -nt 0 -cpo 1aki_md.cpt

2019-02-25 14:08:22,000 [MainThread  ] [INFO ]  Exit code 0

2019-02-25 14:08:22,019 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Markl

0

In [28]:
#Show protein
nglview.show_simpletraj(nglview.SimpletrajTrajectory(output_md_trr_path, output_md_gro_path), gui=True)

NGLWidget(count=4)

In [29]:
#Clean the directory Remove everything
!find . ! \( -iname "*.ipynb" -o -iname "box.png" \) -exec echo rm -rf {} 2>/dev/null \; -print

rm -rf .
.
rm -rf ./.ipynb_checkpoints
./.ipynb_checkpoints
rm -rf ./log.out
./log.out
rm -rf ./log.err
./log.err
rm -rf ./1aki.pdb
./1aki.pdb
rm -rf ./1aki_fixed.pdb
./1aki_fixed.pdb
rm -rf ./1aki_mutated.pdb
./1aki_mutated.pdb
rm -rf ./1aki_pdb2gmx_top.zip
./1aki_pdb2gmx_top.zip
rm -rf ./#1aki_pdb2gmx.gro.1#
./#1aki_pdb2gmx.gro.1#
rm -rf ./1aki_solvate_top.zip
./1aki_solvate_top.zip
rm -rf ./1aki_genion_top.zip
./1aki_genion_top.zip
rm -rf ./#1aki_pdb2gmx.gro.2#
./#1aki_pdb2gmx.gro.2#
rm -rf ./1aki_pdb2gmx.gro
./1aki_pdb2gmx.gro
rm -rf ./#1aki_editconf.gro.1#
./#1aki_editconf.gro.1#
rm -rf ./1aki_editconf.gro
./1aki_editconf.gro
rm -rf ./#1aki_solvate.gro.1#
./#1aki_solvate.gro.1#
rm -rf ./1aki_solvate.gro
./1aki_solvate.gro
rm -rf ./#1aki_gppion.tpr.1#
./#1aki_gppion.tpr.1#
rm -rf ./1aki_gppion.tpr
./1aki_gppion.tpr
rm -rf ./#1aki_genion.gro.1#
./#1aki_genion.gro.1#
rm -rf ./1aki_genion.gro
./1aki_genion.gro
rm -rf ./#1aki_gppmin.tpr.1#
./#1aki_gppmin.tpr.1#
rm -rf ./1aki_gppmin.tpr